In [20]:
from gmail_reader import gmail_authenticate, fetch_all_emails, parse_email

In [10]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import base64, email
from email.utils import parseaddr
from dotenv import load_dotenv
import os

In [22]:
service = gmail_authenticate()

In [ ]:
def fetch_all_emails(service, user_id='me'):
    """
    Fetch all email message IDs from the user's inbox, handling pagination.
    Returns a list of message ID dictionaries.
    """
    all_messages = []
    try:
        # Initial request for the first page of inbox messages (max 500 results)
        response = service.users().messages().list(userId=user_id, labelIds=['INBOX'], maxResults=500).execute()
        messages = response.get('messages', [])
        if messages:
            all_messages.extend(messages)
        # Continue fetching while a next page token exists
        while 'nextPageToken' in response:
            page_token = response['nextPageToken']
            response = service.users().messages().list(
                userId=user_id, labelIds=['INBOX'], maxResults=500, pageToken=page_token
            ).execute()
            messages = response.get('messages', [])
            if messages:
                all_messages.extend(messages)
        return all_messages
    except Exception as e:
        print(f"An error occurred while fetching emails: {e}")
        return []

In [24]:
response = service.users().messages().list(userId='me', labelIds=['INBOX'], maxResults=500).execute()
messages = response.get('messages', [])

RefreshError: ('unauthorized_client: Client is unauthorized to retrieve access tokens using this method, or client not authorized for any of the scopes requested.', {'error': 'unauthorized_client', 'error_description': 'Client is unauthorized to retrieve access tokens using this method, or client not authorized for any of the scopes requested.'})

In [9]:
email_ids = fetch_all_emails(service, user_id='me')
# print(f"Total messages found: {len(email_ids)}")

An error occurred while fetching emails: ('unauthorized_client: Client is unauthorized to retrieve access tokens using this method, or client not authorized for any of the scopes requested.', {'error': 'unauthorized_client', 'error_description': 'Client is unauthorized to retrieve access tokens using this method, or client not authorized for any of the scopes requested.'})


In [ ]:
service = gmail_authenticate()
email_ids = fetch_all_emails(service, user_id='me')
for msg in email_ids:
    msg_id = msg['id']
    details = parse_email(service, msg_id, user_id='me')
    # Print the details for this email
    print(f"From: {details['sender_name']} <{details['sender_email']}>")
    print(f"Date: {details['timestamp']}")
    print(f"Subject: {details['subject']}")
    # Print a snippet of the bodies for brevity
    text_snippet = details['text_body'][:100].strip().replace('\n', ' ')
    html_snippet = details['html_body'][:100].strip().replace('\n', ' ')
    print(f"Body (text): {text_snippet + '...'}")
    if details['html_body']:
        print(f"Body (HTML): {html_snippet + '...'}")
    print(f"Company: {details['company']}")
    print(f"Job Position: {details['job_position']}")
    print(f"Hiring Manager/Recruiter: {details['recruiter_name']}")
    print("-" * 40)

In [29]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]



creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())
else:
    flow = InstalledAppFlow.from_client_secrets_file(
        "credentials.json", SCOPES
    )
    creds = flow.run_local_server(port=0)
# Save the credentials for the next run
with open("token.json", "w") as token:
    token.write(creds.to_json())

# Call the Gmail API
service = build("gmail", "v1", credentials=creds)
results = service.users().labels().list(userId="me").execute()
labels = results.get("labels", [])

if not labels:
    print("No labels found.")
print("Labels:")
for label in labels:
    print(label["name"])


IndentationError: expected an indented block after 'if' statement on line 21 (362849726.py, line 22)

In [30]:
import os.path
from datetime import datetime
import json
import re
import csv
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd
import base64
import json
from bs4 import BeautifulSoup
# from transformers import pipeline

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
  creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
  if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())
  else:
    flow = InstalledAppFlow.from_client_secrets_file(
        "credentials.json", SCOPES
    )
    creds = flow.run_local_server(port=0)
  # Save the credentials for the next run
  with open("token.json", "w") as token:
    token.write(creds.to_json())

# service = build("gmail", "v1", credentials=creds)


FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [ ]:

def extract_plain_text(email_json):
    """
    Extracts and returns only the plain text from a Gmail API email JSON response.
    
    Parameters:
        email_json (dict): JSON response from the Gmail API.
        
    Returns:
        str: Cleaned plain text content of the email.
    """
    payload = email_json.get("payload", {})

    # Check if 'parts' exist (multipart email)
    if "parts" in payload:
        for part in payload["parts"]:
            mime_type = part.get("mimeType", "")
            body_data = part.get("body", {}).get("data", "")

            if body_data:
                decoded_data = base64.urlsafe_b64decode(body_data).decode("utf-8")

                if mime_type == "text/plain":
                    return decoded_data.strip()  # Clean plain text
                elif mime_type == "text/html":
                    return clean_html(decoded_data)  # Convert HTML to plain text

    # If no 'parts', check if 'body' contains the message
    body_data = payload.get("body", {}).get("data", "")
    if body_data:
        decoded_data = base64.urlsafe_b64decode(body_data).decode("utf-8")
        return clean_html(decoded_data)  # Convert HTML to plain text if necessary

    return ""  # Return empty string if no content is found

def clean_html(html_content):
    """
    Removes HTML tags and returns only the plain text content.
    
    Parameters:
        html_content (str): HTML content as a string.
    
    Returns:
        str: Cleaned plain text content.
    """
    soup = BeautifulSoup(html_content, "html.parser")
    text = ' '.join(soup.get_text(separator=' ').split())
    return text.replace('\n', ' ').strip()

num_emails = 200
# get messages
results = service.users().messages().list(userId="me", maxResults=num_emails).execute()
# get list of message ids and thread ids
messages = results.get("messages", [])

# Create empty lists to store data
ids = []
subjects = []
senders = []
sender_names = []
sender_emails = []
snippets = []
bodies = []
dates = []

# get message data using id
for msg in messages:
    msg_id = msg["id"]
    msg_data = service.users().messages().get(userId="me", id=msg_id).execute()
    email_content = extract_plain_text(msg_data)
    # Extract headers
    headers = msg_data["payload"]["headers"]
    # Get info from headers
    subject = next((h["value"] for h in headers if h["name"] == "Subject"), "No Subject")
    sender = next((h["value"] for h in headers if h["name"] == "From"), "Unknown Sender")
    date = next((h["value"] for h in headers if h["name"] == "Date"), "No Date")
    
    # Extract sender name and email if in <> format
    if '<' in sender and '>' in sender:
        sender_name = sender.split('<')[0].strip()
        sender_email = sender[sender.find('<')+1:sender.find('>')]
    else:
        sender_name = ""
        sender_email = sender

    if sender_name.startswith('"') and sender_name.endswith('"'):
        sender_name = sender_name[1:-1]

    # Append data to lists
    ids.append(msg_id)
    subjects.append(subject)
    senders.append(sender)
    sender_names.append(sender_name)
    sender_emails.append(sender_email)
    snippets.append(msg_data.get('snippet', ''))
    bodies.append(email_content)
    # dates.append(msg_data.get('internalDate', ''))
    dates.append(date)

# Create DataFrame from lists
emails_df = pd.DataFrame({
    "Sender Name": sender_names,
    "Sender Email": sender_emails,
    'Subject': subjects, 
    'Body': bodies,
    # 'Date': dates,
    # 'ID': ids,
    # 'Sender': senders,
    # 'Snippet': snippets
    })
emails_df.to_csv("emails.csv", index=False)